# Computational Theory Assessment

In [1]:
import numpy as np

## <Strong>Problem 1 </strong>: Binary Words and Operations

### Parity 
The **Parity** function is defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.1)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as  

$$
\text{Parity}(x, y, z) = x \oplus y \oplus z
$$

where ⊕ shows the **bitwise XOR** operation.

For each bit position, the result is 1 if an **odd number** of x, y and z contain a 1 in that position, otherwise 0.  
Parity therefore acts as an **odd-bit detector**, ensuring that even a single bit change in the inputs alters the output.

**Why XOR is used**
- **XOR** stands for *exclusive OR*. 
- It's a logical operation that compares two bits and outputs **1 if they are different** and **0 if they are the same**.
- XOR is **fast**, **branch-free**, and supported directly by CPUs.  
- It implements addition mod 2 at the bit level.  

**Alternative forms in research**

Some authors and implementations describe *Parity* differently:

| Formulation | Description |
| :-- | :-- |
| `(x + y + z) % 2` | Simplistic show of odd/even nature by summing bits and taking the remainder mod 2. |


**Truth Table (for 1-bit inputs)**

| x | y | z | Parity |
|:--:|:--:|:--:|:--:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 1 |
| 0 | 1 | 1 | 0 |
| 1 | 0 | 0 | 1 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 0 |
| 1 | 1 | 1 | 1 |

In [ ]:
def parity(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """
    Return the parity (XOR) of three 32-bit numbers.

    Each bit of the result is 1 if an odd number of the inputs
    have a 1 in that position, otherwise 0.

    Parameters:
        x (uint32): First 32-bit integer.
        y (uint32): Second 32-bit integer.
        z (uint32): Third 32-bit integer.

    Returns:
        np.uint32: The XOR (parity) of the three input values.
    """
    return np.uint32(x ^ y ^ z)


In [3]:
# quick tests for Parity
print(parity(0, 0, 0))  # expect 0
print(parity(0, 0, 1))  # expect 1
print(parity(1, 1, 0))  # expect 0
print(parity(1, 1, 1))  # expect 1



0
1
0
1


### Choose
The **Choose** function is defined in the [Secure Hash Standard (FIPS 180-4, § 4.1.1)](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as  

$$
\text{Choose}(x, y, z) = (x \land y) \oplus (\lnot x \land z)
$$

**Objective**
Masks the original 32 bit integer by swapping out numbers based on x (similar that X is the key)  
Bitwise NOT operator (~) flips each bit: 0 becomes 1, and 1 becomes 0.

**Explanation**  

**Example**

| x | y | z | Choose |
|:--:|:--:|:--:|:--:|
| 0 | 0 | 0 | 0 |
| 0 | 0 | 1 | 1 |
| 0 | 1 | 0 | 0 |
| 0 | 1 | 1 | 1 |
| 1 | 0 | 0 | 0 |
| 1 | 0 | 1 | 0 |
| 1 | 1 | 0 | 1 |
| 1 | 1 | 1 | 1 |

In [ ]:
def choose(x: np.uint32, y: np.uint32, z: np.uint32) -> np.uint32:
    """
    Return the result of choosing bits from y and z based on x.
    
    For each bit position, if the bit in x is 1, take the bit from y,
    otherwise take the bit from z.

    bitwise NOT operator (~) flips each bit: 0 becomes 1, and 1 becomes 0.
    bitwise AND operator (&) compares each bit of two numbers and returns 1 if both bits are 1, otherwise returns 0.
    bitwise XOR operator (^) compares each bit of two numbers and returns 1 if the bits are different, otherwise returns 0.

    Parameters:
        x (uint32): The mask.
        y (uint32): The first number.
        z (uint32): The second number.
    """
    return np.uint32((x & y) ^ (~x & z))


In [5]:
# quick tests for Choose
print(choose(np.uint32(0), np.uint32(0), np.uint32(0)))   # expect 0
print(choose(np.uint32(0), np.uint32(0), np.uint32(1)))   # expect 1
print(choose(np.uint32(1), np.uint32(1), np.uint32(0)))   # expect 1
print(choose(np.uint32(1), np.uint32(1), np.uint32(1)))   # expect 1

# small demo with numbers
print(choose(np.uint32(0), np.uint32(5), np.uint32(10)))        # expect 10
print(choose(np.uint32(0xFFFFFFFF), np.uint32(5), np.uint32(10)))  # expect 5


0
1
1
1
10
5


## <strong>Problem 2 </strong>: Fractional Parts of Cube Roots

## <strong>Problem 3 </strong>: Padding

## <strong>Problem 4 </strong>: Hashes

## <strong>Problem 5 </strong>: Passwords